Name: Yiğit Özgümüş

I hereby declare that I observed the honour code of the university when preparing the homework.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display, Math, Latex
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "last_expr"

## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [2]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']
test_strings

['th__br__n.f_x.',
 '_u_st__n_.to_be._nsw_r__',
 'i__at_._a_h_n_._e_r_i_g',
 'q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [3]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}
idx2letter = {i:c for i,c in enumerate(alphabet)}
T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


Reformat data for better visualization

In [4]:
T_new = np.matrix(T).reshape(27,27);
import string
data1 =pd.DataFrame(data=T_new,index=list(alphabet),columns=list(alphabet));
data =data1.apply(pd.to_numeric,errors='coerce')
# P(x_t = 'u'| x_t-1 = 'q')
data['u']['q']
data.head()


0.9949749

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,.
a,0.000284,0.022830,0.036904,0.042629,0.001222,0.007574,0.017138,0.001466,0.037266,0.000235,...,0.110477,0.093429,0.131796,0.009803,0.030657,0.008880,0.000956,0.023370,0.001870,0.071522
b,0.058003,0.005870,0.000079,0.002263,0.341671,0.000206,0.000427,0.000364,0.047908,0.007689,...,0.074030,0.022688,0.010743,0.119613,0.001155,0.000032,0.000000,0.086450,0.000000,0.007452
c,0.122984,0.000027,0.021545,0.000525,0.171592,0.000009,0.000000,0.170172,0.056549,0.000000,...,0.037681,0.001049,0.090676,0.035836,0.000000,0.000000,0.000000,0.004197,0.000009,0.015177
d,0.028035,0.000506,0.000258,0.008109,0.122483,0.000680,0.005484,0.000708,0.079490,0.000348,...,0.017892,0.030704,0.000916,0.017881,0.002776,0.001365,0.000000,0.007648,0.000000,0.620154
e,0.054587,0.001280,0.022432,0.084343,0.031710,0.008564,0.005283,0.001776,0.012719,0.000260,...,0.130752,0.071279,0.024154,0.001429,0.015731,0.007088,0.010514,0.012600,0.000183,0.352561


### Task 1

We can rewrite the probability formula $p(x_{1:N}|x_0)$ as 
$$ p(x_{1:N}\mid x_0=\hat{x_0}) \sim P(x_1\mid x_0=\hat{x_0}) P(x_2\mid x_1) P(x_3\mid x_2)...P(x_n\mid x_n-1)$$

Then we can choose each letter using the np.random.choice function iterating over the lenght of the desired string

In [5]:
#Consider x0 as letter .
def get_sample(N, x0 = '.'):
    result = []
    result.append(x0) # x0 choice
    for i in range(N):
        prob = data.loc[result[-1]]
        prob_norm = prob / prob.sum(axis=0)
        choice = np.random.choice(alphabet, p = list(prob_norm))
        result.append(choice)
    return ''.join(result[1:])
        

In [53]:
for i in range(5):
    get_sample(50)

'..ts.r..hat.jalye.h.tllindreed.inend.ead.alo.ps.pe'

'thince..wemyouly.ad.ig.hefack.osan.ad.d.t.hat.joor'

'and.p.sthacee..tarinth.thend.steinymincas.mureddis'

'.abl.ve.y.tof.t.hun.chatonenousellnol.nto.gr.mpe.e'

'e.urgealan.butataf.fasme.pit.bjavaroth.leviningedo'

### Task 2

Let's assume that we have string "a__t" which has 2 unknown values. Under the Markov(l) model assumption, for the second and third letter, we have the last known letter which is first and the first known letter after the unknowns which is the fourth to use in the distribution decision. For this problem, I assumed that the first and the last letter of every given sentence is known so I added colons to each side of the sentences. Then if we have a given string length of N, after the addition of colons, the problem transforms into something like below:

$$p(x_{1:N}\mid x_0 = \hat{x_0},x_{n+1}=\hat{x_{n+1}}) \sim p(x_{n+1}=\hat{x_{n+1}}\mid x_{n}) p(x_{n}\mid x_{n-1})p(x_{n-1}\mid x_{n-2})p(x_{n-2}\mid x_{n-3})...p(x_{2}\mid x_{1})p(x_{1}\mid x_{0}=\hat{x_0})$$

We can obtain the last probability in the iteration directly from the transition matrix, for the rest of the bundle, we can take the $N'th$ power of the transition matrix and get the distribution vector regarding the $x_{n+1}$. Elementwise multiplication of these two vectors will give us the probability vector we want. Then we can normalize this vector and we can use np.random.choice method again to predict the missing letter. This strategy will be used iteratively from the start of the string and each predicted letter will be contributed to the calculation in the next iteration.

In [50]:
def make_dataframe(matrix):
    new_df =pd.DataFrame(data=matrix,index=list(alphabet),columns=list(alphabet));
    new_df =new_df.apply(pd.to_numeric,errors='coerce')
    return new_df

def normalize(vector):
    return vector / vector.sum(axis=0)

def guess_the_blank(index,sentence,probability):
    arrayified = list(sentence)
    new_guess = np.random.choice(alphabet,p=probability)
    arrayified[index] = new_guess
    return ''.join(arrayified)

def fill_the_blanks(sentence):
    sentence = "." + sentence + "."
    #print(sentence)
    while ("_" or "?") in sentence:
        index = sentence.index("_" or "?")
        prior_index = index-1
        pr_letter = sentence[prior_index]
        #print("pri: ",pr_letter)
        post_index = 0
        for k in range(index,len(sentence)):
         #   print("k",k)
          #  print("letter is: ",sentence[k])
            if(sentence[k] != "_"):
                post_index = k 
                break
        #print("------")
        po_letter = sentence[post_index]
       # print("po: ",po_letter)
        length = post_index - prior_index-1
        powered = np.linalg.matrix_power(data.as_matrix(),length)
        powered_df =make_dataframe(powered)
        probability = normalize(powered_df[po_letter] * data.loc[pr_letter])
        sentence = guess_the_blank(index,sentence,probability)
        #print("guessed sentence: ",sentence[1:-1])
    return sentence[1:-1]

for i in range(len(test_strings)):
    print("Test String " + str(i+1),": ", test_strings[i])
    for j in range(10):
        print("Sample ",str(j+1), "=", fill_the_blanks(test_strings[i]))

Test String 1 :  th__br__n.f_x.
Sample  1 = the.brean.fex.
Sample  2 = tht.brion.fax.
Sample  3 = the.br..n.fix.
Sample  4 = thembrton.fex.
Sample  5 = the.br.an.fex.
Sample  6 = the.brsan.fex.
Sample  7 = the.br.on.fex.
Sample  8 = thambryon.fex.
Sample  9 = the.brean.fex.
Sample  10 = the.brd.n.fix.
Test String 2 :  _u_st__n_.to_be._nsw_r__
Sample  1 = ouesthend.tombe.inswar.s
Sample  2 = mubsthane.tombe.answer.t
Sample  3 = ouisthand.to.be.answer.f
Sample  4 = musst.ang.tombe.answaris
Sample  5 = ou.stignn.to.be.onswer.s
Sample  6 = sunsthiny.to.be.enswered
Sample  7 = cussthend.to.be.answerin
Sample  8 = ouist.ing.to.be.onswer.h
Sample  9 = mussttinf.toube.inswar.d
Sample  10 = bulsthond.tombe..nswor.s
Test String 3 :  i__at_._a_h_n_._e_r_i_g
Sample  1 = is.ate.washene.meerwing
Sample  2 = io.ath.bathong.rearaing
Sample  3 = is.ats.sa.hand.te.rdi.g
Sample  4 = ighats.ma.hend.petrding
Sample  5 = ielaty.rathind.setr.ing
Sample  6 = in.atl.tatheng.jear.ing
Sample  7 = icuatt.hathend.

### Task 3

The same solution strategy at the task 2 is used here. The only difference is instead of choosing with np.random.choice, we choose the most likely one, in this case the letter with the highest probability. The summation of the log probabilities are also shown for each test string.

In [51]:
def make_dataframe(matrix):
    new_df =pd.DataFrame(data=matrix,index=list(alphabet),columns=list(alphabet));
    new_df =new_df.apply(pd.to_numeric,errors='coerce')
    return new_df

def normalize(vector):
    return vector / vector.sum(axis=0)

def guess_the_blank_mle(index,sentence,probability):
    arrayified = list(sentence)
    new_guess = np.argmax(probability)
    arrayified[index] = new_guess
    log_prob = np.log(probability[new_guess])
    return ''.join(arrayified),log_prob

def fill_the_blanks3(sentence):
    sentence = "." + sentence + "."
    log_probability = 0
    #print(sentence)
    while ("_" or "?") in sentence:
        index = sentence.index("_" or "?")
        prior_index = index-1
        pr_letter = sentence[prior_index]
        #print("pri: ",pr_letter)
        post_index = 0
        for k in range(index,len(sentence)):
         #   print("k",k)
          #  print("letter is: ",sentence[k])
            if(sentence[k] != "_"):
                post_index = k 
                break
        #print("------")
        po_letter = sentence[post_index]
       # print("po: ",po_letter)
        length = post_index - prior_index-1
        powered = np.linalg.matrix_power(data.as_matrix(),length)
        powered_df =make_dataframe(powered)
        probability = normalize(powered_df[po_letter] * data.loc[pr_letter])
        sentence,temp_log_prob = guess_the_blank_mle(index,sentence,probability)
        log_probability += temp_log_prob
        #print("guessed sentence: ",sentence[1:-1])
    return sentence[1:-1], log_probability

for i in range(len(test_strings)):
    print("Test String " + str(i+1),": ", test_strings[i])
    prediction, logP = fill_the_blanks3(test_strings[i])
    print("Prediction ", "=", prediction)
    print("Log probability: ",logP,"\n" )


Test String 1 :  th__br__n.f_x.
Prediction  = the.br.an.fex.
Log probability:  -3.07433486577 

Test String 2 :  _u_st__n_.to_be._nsw_r__
Prediction  = oursthend.to.be.answered
Log probability:  -11.0693279723 

Test String 3 :  i__at_._a_h_n_._e_r_i_g
Prediction  = in.ath.wathend.he.r.ing
Log probability:  -11.636089996 

Test String 4 :  q___t.___z._____t.__.___.__.
Prediction  = qur.t.thiz.the.at.an.the.an.
Log probability:  -22.9236424228 



### Task 4

Considering the language, higher Markov model could give us better estimation in terms of similarity to english language. The 1 letter distance in the current model occasionaly can create meaningful words such as "the" with randomization approach. It gave relatively better results with most likely letter esti, but with the increased level of distance in the model, we can estimate better values and obtain a better similarity. But increasing the Markov model will also increase the complexity of the computation model itself. So there will be a certain trade off to consider.